<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-задания-нужно-выполнить-только-с-помощью-sql" data-toc-modified-id="Все-задания-нужно-выполнить-только-с-помощью-sql-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все задания нужно выполнить только с помощью sql</a></span></li></ul></div>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [6]:
data = pd.read_csv('./csv_for_sql/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [7]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [8]:
pd.read_sql(
    '''
    SELECT count(*) FROM boston
    ''',
    con,
)

,count(*)
0,506


In [26]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [27]:
tasks+=1
pd.read_sql(
    '''
    select 
        'CRIM' as Column, sum(case when CRIM is null then 1 else 0 end) as empty_cnt
    FROM boston    
    union all
    select
        'ZN' as Column, sum(case when ZN is null then 1 else 0 end)as empty_cnt
    FROM boston    
    union all
    select
        'INDUS' as Column, sum(case when  INDUS is null then 1 else 0 end) as empty_cnt
    FROM boston
    union all
    select
        'CHAS' as Column, sum(case when CHAS is null then 1 else 0 end) as empty_cnt
    FROM boston
    union all
    select
        'NOX' as Column, sum(case when NOX is null then 1 else 0 end) as empty_cnt
    FROM boston
    ''',
    con,
)

,Column,empty_cnt
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [28]:
tasks+=1
pd.read_sql(
    '''
        SELECT 
           'CRIM' as Column, count(distinct(CRIM)) as unique_cnt
        FROM boston    
        union all
        select   
           'ZN' as Column, count(distinct(ZN)) as unique_cnt
        FROM boston    
        union all
        select   
           'INDUS' as Column,  count(distinct(INDUS)) as unique_cnt
        FROM boston    
        union all
        select   
           'CHAS' as Column,  count(distinct(CHAS)) as unique_cnt
        FROM boston    
        union all
        select   
           'NOX' as Column,  count(distinct(NOX)) as unique_cnt
        FROM boston
    ''',
    con,
)

,Column,unique_cnt
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [29]:
tasks+=1
pd.read_sql(
    '''
    with medians as (
        SELECT 'CRIM' as COLUMN_NAME, AVG(CRIM) as median_value
        FROM (
                SELECT CRIM
                FROM boston
                ORDER BY CRIM
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'ZN' as COLUMN_NAME, AVG(ZN) as median_value
        FROM (
                SELECT ZN
                FROM boston
                ORDER BY ZN
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'INDUS' as COLUMN_NAME, AVG(INDUS) as median_value
        FROM (
                SELECT INDUS
                FROM boston
                ORDER BY INDUS
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'CHAS' as COLUMN_NAME, AVG(CHAS) as median_value
        FROM (
                SELECT CHAS
                FROM boston
                ORDER BY CHAS
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'NOX' as COLUMN_NAME, AVG(NOX) as median_value
        FROM (
                SELECT NOX
                FROM boston
                ORDER BY NOX
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)))
    , mins as(
        SELECT distinct
               'CRIM' as COLUMN_NAME
                , min(CRIM) OVER() as min_value
        FROM boston b
        UNION ALL
        SELECT distinct
               'ZN' as COLUMN_NAME
                , min(ZN) OVER() as min_value
        FROM boston b
        UNION ALL
        SELECT distinct
               'INDUS' as COLUMN_NAME                
                , min(INDUS) OVER() as min_value
        FROM boston b
        UNION ALL
        SELECT distinct
               'CHAS' as COLUMN_NAME
                , min(CHAS) OVER() as min_value
        FROM boston b
        UNION ALL
        SELECT distinct
               'NOX' as COLUMN_NAME
                , min(NOX) OVER() as min_value
        FROM boston b 
    )
    
    SELECT distinct
           mins.COLUMN_NAME
            , mins.min_value
            , medians.median_value
    FROM mins  
        join medians on medians.COLUMN_NAME=mins.COLUMN_NAME
    where min_value=median_value
        
    ''',
    con,
)

,COLUMN_NAME,min_value,median_value
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


Выводы: 

Не менее чем в половине рассматриваемых кейсов доля земель под жилую застройку не зонирована под участки площадью более 25 000 кв. футов.

Не менее чем в половине рассматриваемых кейсов участки не граничат с рекой.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [30]:
tasks+=1
pd.read_sql(
    '''
     with tmp as (
         select MEDV
                ,RM
                , avg(RM) FILTER(where MEDV in (select MAX(MEDV) FROM boston)) over() as RM_AvgMax 
                , row_number() over(ORDER BY MEDV asc) as RowNumber
         FROM boston )
    , counts as ( 
         select column1 as FirstRowCount 
         FROM (VALUES (25), (50), (100), (200), (300))
         )
     
     select t.* 
           , RM_AvgMax - RM_Avg as Difference
     from (
           select distinct FirstRowCount
                  , avg(RM) over(partition by FirstRowCount) as RM_Avg
                  , RM_AvgMax
          from counts 
               join tmp on tmp.RowNumber<=counts.FirstRowCount) t
    ''',
    con,
)

,FirstRowCount,RM_Avg,RM_AvgMax,Difference
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


С увеличением количества комнат наблюдаестя увеличение стоимости дома.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [38]:
tasks+=1
pd.read_sql(
    '''
    select * from(
        select LSTAT, MEDV
            , rank() over (ORDER BY LSTAT asc) as Rank
        FROM boston)
    where MEDV in (select MAX(MEDV) FROM boston)
    ''',
    con,
)

,LSTAT,MEDV,Rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Вывод: Разброс положений этих домов в рейтинге довольно большой. Значит, показатели "процент населения с более низким статусом" и "стоимость дома" слабо связаны между собой. 

С точки зрения потенциального покупателя высокая дома стоимость не гарантирует низкое значение процента населения с более низким статусом в доме.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [32]:
tasks+=1
pd.read_sql(
    '''
    SELECT CHAS, avg(MEDV) 
    FROM boston
    group by CHAS
    ''',
    con,
)

,CHAS,avg(MEDV)
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [36]:
tasks+=1
pd.read_sql(
    '''
    with tmp as (
                SELECT distinct 'CRIM' as column_name
                        , avg(CRIM) FILTER(where CHAS = 1.0) over()  avgCHAS_1
                        , avg(CRIM) FILTER(where CHAS = 0.0) over()  avgCHAS_0 
                FROM boston
                
                UNION ALL
                SELECT distinct 'ZN' as column_name
                        , avg(ZN) FILTER(where CHAS = 1.0) over()  avgCHAS_1
                        , avg(ZN) FILTER(where CHAS = 0.0) over()  avgCHAS_0 
                FROM boston
                
                UNION ALL
                SELECT distinct 'INDUS' as column_name
                        , avg(INDUS) FILTER(where CHAS = 1.0) over()  avgCHAS_1
                        , avg(INDUS) FILTER(where CHAS = 0.0) over()  avgCHAS_0 
                FROM boston
                
                UNION ALL
                SELECT distinct 'NOX' as column_name
                        , avg(NOX) FILTER(where CHAS = 1.0) over()  avgCHAS_1
                        , avg(NOX) FILTER(where CHAS = 0.0) over()  avgCHAS_0 
                FROM boston
                )
    select * from tmp where avgCHAS_1>avgCHAS_0
    ''',
    con,
)

,column_name,avgCHAS_1,avgCHAS_0
0,INDUS,12.719143,11.019193
1,NOX,0.593426,0.551817


Вывод:
    Когда дом граничит с рекой значения выше для следующих показателей:
             - доля акров неторгового бизнеса на город;      
             - концентрация оксидов азота.        
             
Из чего можно предположить, что в этих районах более распространена промышленность, чем в не граничащих с рекой районах.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [34]:
tasks+=1
pd.read_sql(
    '''
     with 
       perc as ( 
         select column1 as Perc
         , round(cnt*column1) as idx
         FROM (VALUES (0.1), (0.2), (0.3), (0.4), (0.5), (0.6), (0.7), (0.8), (0.9), (1))
             join (select count(*) cnt from boston)
         )
      ,t_rating as (
        select INDUS, NOX, i.RowNumber from (
                    select INDUS, row_number() over(ORDER BY INDUS asc) as RowNumber
                    from boston) i
                join (
                    select NOX,row_number() over(ORDER BY NOX asc) as RowNumber
                    from boston) n 
                on n.RowNumber=i.RowNumber
                )
    select CAST(Perc*100 AS INT) Perc
          ,INDUS
          ,NOX
    from perc
        join t_rating on idx=RowNumber
    ''',
    con,
)

,Perc,INDUS,NOX
0,10,2.89,0.426
1,20,4.15,0.442
2,30,5.96,0.472
3,40,7.07,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


In [37]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
